## **Naver Map Crawling with Python Selenium**

- 준비물: Chrome, ChromeDriver, Selenium
- 참고  
https://velog.io/@kimdy0915/Selenium-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%B9-%ED%8E%98%EC%9D%B4%EC%A7%80-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0%EC%84%B8%ED%8C%85

In [ ]:
import json
import re
import time
import random
import pandas as pd
from tqdm import tqdm

from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [5]:
# 드라이버 경로
driver_path = r'C:\Users\scsi\Desktop\오세훈\SCSI\2024-2\국토부\chromedriver-win64\chromedriver.exe'

# 드라이버 초기화
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

In [6]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

def safe_get_texts(element, css_selector, default="정보 없음"):
    """
    요소에서 텍스트를 안전하게 추출하는 함수.
    - 요소가 하나면 단일 텍스트를 반환하고,
    - 여러 개면 텍스트 리스트를 반환합니다.
    """
    try:
        # 모든 요소 찾기
        elements = element.find_elements(By.CSS_SELECTOR, css_selector)
        
        if not elements:
            return default  # 요소를 찾지 못한 경우 기본값 반환
        elif len(elements) == 1:
            return elements[0].text  # 요소가 하나일 경우 텍스트 반환
        else:
            return [el.text for el in elements]  # 여러 요소가 있을 경우 텍스트 리스트 반환

    except:
        return default

def extract_cafe_info(driver, max_attempts=5):
    """
    카페의 주요 정보를 추출하는 함수.
    이름, 유형, 리뷰 수, 주소, 영업시간, 포장/반려동물 여부, 메뉴를 통합적으로 추출합니다.
    스크롤을 내려가며, 데이터를 찾지 못한 경우 누락된 데이터만 다시 시도합니다.

    Returns:
        dict: 추출된 카페 정보 딕셔너리.
    """
    business_hours = []
    menus = []
    packaging_available = None
    pet_friendly = None

    # 기본 정보 초기화
    name = cafe_type = reviews = address = None

    # 1. 기본 정보 수집 (이름, 유형, 리뷰 수, 주소)
    try:
        name = WebDriverWait(driver, 10).until(
            lambda d: safe_get_texts(d, '.GHAhO')
        )  # 카페 이름이 로드될 때까지 대기
        cafe_type = WebDriverWait(driver, 10).until(
            lambda d: safe_get_texts(d, '.lnJFt')
        )  # 카페 유형이 로드될 때까지 대기
        reviews = WebDriverWait(driver, 10).until(
            lambda d: safe_get_texts(d, '.PXMot')
        )  # 리뷰 수가 로드될 때까지 대기
        address = WebDriverWait(driver, 10).until(
            lambda d: safe_get_texts(d, 'span.LDgIH')
        )  # 카페 주소가 로드될 때까지 대기

        # 영업 시간 정보 수집
        expand_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.gKP9i.RMgN0'))
        )
        expand_button.click()
        days_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.w9QyJ'))
        )
        for day_element in days_elements:
            day_name = safe_get_texts(day_element, 'span.i8cJw', "알 수 없는 요일")
            hours_info = safe_get_texts(day_element, 'div.H3ua4', "정보 없음")
            if day_name != "알 수 없는 요일":
                business_hours.append({'day': day_name, 'hours': hours_info})
    except Exception as e:
        print("기본 정보를 추출하는 동안 오류 발생:", e)

    # 2. 메뉴 및 포장 가능 여부 정보 수집 (스크롤 반복)
    attempt = 0
    while attempt < max_attempts:
        # 메뉴 정보 수집
        if not menus:
            menu_selectors = [
                ('li.ipNNM', 'span.VQvNX'),  # 우선순위 메뉴
                ('li.gHmZ_', 'a.ihmWt')      # 대체 메뉴
            ]
            for item_selector, text_selector in menu_selectors:
                menu_items = driver.find_elements(By.CSS_SELECTOR, item_selector)
                if menu_items:
                    menus = [
                        safe_get_texts(item, text_selector, default="메뉴 정보 없음")
                        for item in menu_items
                    ]
                    break  # 첫 번째로 발견된 메뉴 리스트만 사용

        # 포장 가능 여부 및 반려동물 동반 여부 확인
        if packaging_available is None and pet_friendly is None:
            features_text = safe_get_texts(driver, 'div.xPvPE')
            packaging_available = "포장" in features_text
            pet_friendly = "반려동물 동반" in features_text

        # 모든 정보가 수집되면 반복을 종료
        if menus and (packaging_available is not None and pet_friendly is not None):
            break

        # 필요한 정보를 모두 수집하지 못했다면 스크롤을 내리고 다시 시도
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 로딩을 기다리기 위해 2초 대기
        attempt += 1  # 시도 횟수 증가

    # 기본값 설정 (수집되지 않은 정보가 있을 경우)
    if not business_hours:
        business_hours = [{"day": "정보 없음", "hours": "정보 없음"}]
    if not menus:
        menus = ["메뉴 정보 없음"]
    if packaging_available is None:
        packaging_available = False
    if pet_friendly is None:
        pet_friendly = False

    # 결과를 딕셔너리 형태로 반환
    return {
        "name": name,                     # 카페 이름
        "type": cafe_type,                # 카페 유형
        "reviews": reviews,               # 리뷰 수
        "address": address,               # 카페 주소
        "business_hours": business_hours, # 요일별 영업 시간
        "menus": menus,                   # 메뉴 리스트
        "packaging_available": packaging_available, # 포장 가능 여부
        "pet_friendly": pet_friendly      # 반려동물 동반 가능 여부
    }

<p align="center">
  <img src="./figure/data_description.png" alt="Data Description">
</p>

- Chrome 창이 열리면 F12 개발자 도구를 열고, 필요한 정보에 대한 CSS 클래스명을 찾은 다음 코드를 작성한다.

In [7]:
url = 'https://map.naver.com/v5/search'
driver.get(url)
key_word = '서초구 카페'    # 검색어

# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(3)

In [ ]:
# 전체 데이터를 저장할 리스트
all_names = []
all_types = []
all_n_reviews = []
all_addresses = []
all_menus = []
all_business_hours = []
all_packaging = []
all_pet_friendly = []

# 전체 페이지 수 가져오기
next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F > a')
num_pages = len(next_btn)

# 페이지 리스트 반복
for page_num in range(1, num_pages):
    print(f"페이지 진행 상황: {page_num}/{num_pages}")  # 페이지 진행 상황 표시

    # 현재 페이지의 카페 리스트 추출 및 필터링
    cafe_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')
    filtered_cafe_list = [cafe for cafe in cafe_list if "UEzoS rTjJo" in cafe.get_attribute("class") and "cZnHG" not in cafe.get_attribute("class")]

    # 카페 진행 상황을 tqdm으로 시각화
    for cafe in tqdm(filtered_cafe_list, desc="카페 진행 상황", leave=False):
        try:
            # 카페 클릭 및 대기 시간 최적화
            WebDriverWait(cafe, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.tzwk0'))).click()
            time.sleep(random.uniform(1, 2))  # 대기 시간을 최소화

            # 카페 정보 추출
            switch_frame('entryIframe')
            cafe_info = extract_cafe_info(driver)
            time.sleep(random.uniform(1, 5))

            # 각 정보 리스트에 추가
            all_names.append(cafe_info["name"])
            all_types.append(cafe_info["type"])
            all_n_reviews.append(cafe_info["reviews"])
            all_addresses.append(cafe_info["address"])
            all_business_hours.append(cafe_info["business_hours"])
            all_packaging.append(cafe_info["packaging_available"])
            all_pet_friendly.append(cafe_info["pet_friendly"])
            all_menus.append(cafe_info["menus"])

        except Exception as e:
            print(f"오류 발생: {e}")  # 오류 시 로그를 간결하게 남깁니다.
            continue

        finally:
            # 검색 프레임으로 복귀
            switch_frame('searchIframe')

    if not next_btn[-1].is_enabled():
        break  # 페이지 넘김 버튼이 비활성화되어 있을 경우 반복 종료

    # 다음 페이지로 이동
    next_btn[-1].click()
    time.sleep(random.uniform(2, 3))  # 페이지 로딩 최적화

# 결과물을 DataFrame으로 변환 및 저장
data = pd.DataFrame({
    "이름": all_names,
    "유형": all_types,
    "리뷰 수": all_n_reviews,
    "주소": all_addresses,
    "영업 시간": all_business_hours,
    "포장 가능 여부": all_packaging,
    "반려동물 동반 가능 여부": all_pet_friendly,
    "메뉴": all_menus
})

data.to_csv("cafe_data_1112.csv", index=False, encoding="utf-8-sig")